In [1]:
import pandas as pd

df = pd.read_csv('../data/game_data.csv')

df = df.drop(columns=['name','scrape_id', 'main_story_polled', 'main_+_extras_polled', 'completionist_polled', 'all_playstyles_polled', 'developers', 'publishers', 'aliases', 'alias'])

df = df.dropna(subset=['genres'])
df = df.dropna(subset=['platforms'])

df['rating'] = pd.to_numeric(df['rating'], errors="coerce")
df = df.dropna(subset=['rating'])
df['rating'] = df['rating'] * 100
df['rating'] = df['rating'].astype(int)


df = df.dropna(subset=['na', 'eu', 'jp'], how='all')
df['eu'] = df['eu'].fillna(df['na'])
df['jp'] = df['jp'].fillna(df['na'])
df['eu'] = df['eu'].fillna(df['jp'])
df['jp'] = df['jp'].fillna(df['eu'])
df['na'] = df['na'].fillna(df['eu'])
df['eu'] = pd.to_datetime(df['eu'])
df['na'] = pd.to_datetime(df['na'])
df['jp'] = pd.to_datetime(df['jp'])

df = df.dropna(subset=['developer', 'publisher'], how='all')
df['developer'].fillna(df['publisher'], inplace=True)
df['publisher'].fillna(df['developer'], inplace=True)

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        df[col] = pd.to_timedelta(df[col], errors="coerce")

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        df[col].fillna(df[col].mean(), inplace=True)

missing_values = df.isnull().sum()
print(missing_values)
df.count
# len(df.columns)


rating                    0
beat_count                0
platforms                 0
genres                    0
developer                 0
publisher                 0
na                        0
main_story_average        0
main_story_median         0
main_story_rushed         0
main_story_leisure        0
main_+_extras_average     0
main_+_extras_median      0
main_+_extras_rushed      0
main_+_extras_leisure     0
completionist_average     0
completionist_median      0
completionist_rushed      0
completionist_leisure     0
all_playstyles_average    0
all_playstyles_median     0
all_playstyles_rushed     0
all_playstyles_leisure    0
eu                        0
jp                        0
dtype: int64


<bound method DataFrame.count of        rating  beat_count                                          platforms  \
0          75         152                                       ['Xbox 360']   
1          76         225  ['Nintendo Switch', 'PC', 'PlayStation 2', 'Pl...   
2          79         135           ['PC', 'PlayStation 2', 'PlayStation 4']   
3          78         106           ['PC', 'PlayStation 2', 'PlayStation 4']   
4          73         308                                  ['PlayStation 2']   
...       ...         ...                                                ...   
10776      66         233  ['PC', 'PlayStation 4', 'PlayStation 5', 'Xbox...   
10777      76         103  ['PC', 'PlayStation 4', 'PlayStation 5', 'Xbox...   
10778      72          35         ['Nintendo Switch', 'PC', 'PlayStation 4']   
10779      76          43                                             ['PC']   
10780      61          41  ['Mac', 'Mobile', 'Nintendo Switch', 'PC', 'Pl...   

      

In [2]:
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer

df["genres"] = df["genres"].apply(literal_eval)
df["platforms"] = df["platforms"].apply(literal_eval)

mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df['genres']), columns=mlb.classes_, index=df.index)

mlb = MultiLabelBinarizer()
platforms_encoded = pd.DataFrame(mlb.fit_transform(df['platforms']), columns=mlb.classes_, index=df.index)


developer_counts = df['developer'].value_counts()
publisher_counts = df['publisher'].value_counts()

threshold = 5

selected_developers = developer_counts[developer_counts > threshold].index
selected_publishers = publisher_counts[publisher_counts > threshold].index

df_encoded_dev = pd.get_dummies(df, columns=['developer'])
df_encoded_pub = pd.get_dummies(df, columns=['publisher'])

df_encoded_dev = df_encoded_dev.drop(columns=set(df_encoded_dev.columns) - set(['developer_' + dev for dev in selected_developers]))
df_encoded_pub = df_encoded_pub.drop(columns=set(df_encoded_pub.columns) - set(['publisher_' + pub for pub in selected_publishers]))

data_encoded = pd.concat([df, df_encoded_dev, df_encoded_pub, genres_encoded, platforms_encoded], axis=1)

data_encoded['na_year'] = data_encoded['na'].dt.year
data_encoded['na_month'] = data_encoded['na'].dt.month
data_encoded['na_day'] = data_encoded['na'].dt.day

data_encoded['eu_year'] = data_encoded['eu'].dt.year
data_encoded['eu_month'] = data_encoded['eu'].dt.month
data_encoded['eu_day'] = data_encoded['eu'].dt.day

data_encoded['jp_year'] = data_encoded['jp'].dt.year
data_encoded['jp_month'] = data_encoded['jp'].dt.month
data_encoded['jp_day'] = data_encoded['jp'].dt.day


data_encoded = data_encoded.drop(columns=["genres", "platforms", "developer", "publisher", "na", "jp", "eu"])

for col in data_encoded.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        data_encoded[col] = data_encoded[col].dt.total_seconds() / 60



In [3]:
data_encoded

,rating,beat_count,main_story_average,main_story_median,main_story_rushed,main_story_leisure,main_+_extras_average,main_+_extras_median,main_+_extras_rushed,main_+_extras_leisure,...,Zeebo,na_year,na_month,na_day,eu_year,eu_month,eu_day,jp_year,jp_month,jp_day
0,75,152,467.0,480.0,371.0,566.0,554.0,600.0,378.0,888.0,...,0,2009,7,22,2009,7,22,2009,7,22
1,76,225,1150.0,1129.0,779.0,1889.0,1506.0,1440.0,1066.0,2503.0,...,0,2006,10,24,2006,10,24,2006,5,18
2,79,135,1206.0,1213.0,878.0,1715.0,1991.0,1800.0,1419.0,3238.0,...,0,2007,5,8,2007,5,8,2006,9,28
3,78,106,1291.0,1260.0,911.0,1769.0,2009.0,1909.0,1620.0,2728.0,...,0,2008,3,25,2008,3,25,2007,1,18
4,73,308,1025.0,1025.0,751.0,1295.0,1248.0,1200.0,923.0,1840.0,...,0,2003,2,11,2003,2,11,2003,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,66,233,423.0,420.0,330.0,510.0,563.0,514.0,458.0,839.0,...,0,2022,11,18,2022,11,18,2022,11,18
10777,76,103,1976.0,1890.0,1656.0,2598.0,2889.0,2760.0,2352.0,3729.0,...,0,2022,10,27,2022,10,27,2022,10,27
10778,72,35,74.0,70.0,58.0,86.0,98.0,100.0,79.0,115.0,...,0,2021,9,24,2021,9,24,2021,9,24
10779,76,43,131.0,120.0,107.0,170.0,135.0,125.0,120.0,155.0,...,0,2021,11,1,2021,11,1,2021,11,1
